In [ ]:
# ✅ Install all required libraries for Gemini + LangChain + Google Sheets
!pip install \
    google-generativeai \
    langchain-google-genai \
    langchain_experimental \
    gspread \
    python-dotenv \
    oauth2client


In [ ]:
GOOGLE_API_KEY= "xxxxxxxxxxxxxxxxxxxxxxxxx"


In [ ]:
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

import google.generativeai as gemini
gemini.api_key = os.getenv("GOOGLE_API_KEY")


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    google_api_key=os.getenv("GOOGLE_API_KEY")
)


In [ ]:
# ----------------------------------------
# 2. Import Libraries
# ----------------------------------------
import os
import gspread
import pandas as pd
from dotenv import load_dotenv
from oauth2client.service_account import ServiceAccountCredentials

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_experimental.tools.python.tool import PythonREPLTool

# ----------------------------------------
# 3. Load Environment Variables
# ----------------------------------------
load_dotenv()
gemini_api_key = os.getenv("GOOGLE_API_KEY")

# ----------------------------------------
# 4. Connect to Google Sheets and Load Data
# ----------------------------------------
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("xxxxxxxxxxxxxxxxxxx", scope)
client = gspread.authorize(creds)
sheet = client.open_by_key("xxxxxxxxxxxxxxxxxxx").sheet1
data = pd.DataFrame(sheet.get_all_records())

# ----------------------------------------
# 7. Create LangChain Agent for DataFrame
# ----------------------------------------
pandas_agent = create_pandas_dataframe_agent(
    llm=llm,
    df=data,
    verbose=True,
    agent_type="openai-tools",  # Gemini works with this type
    allow_dangerous_code=True
)

In [ ]:
response = pandas_agent.run("The dataframe `df` has a column called `date` (datetime), a column called `type`, and a column called `amount`.Filter the dataframe to include:- rows where the `date` is between January 1st 2025 and March 31st 2025- and the `type` is equal to cost.Then calculate the **total** of the `amount` column for these rows.Store the result in a variable called `result` and print it:The total cost between January 1st 2025 and March 31st 2025 is {result:,} UGX ")
print(response)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "import pandas as pd\n\ndf_dates = pd.to_datetime(df['Date'])\n\ndf_filtered = df[(df_dates >= pd.to_datetime('2025-01-01')) & (df_dates <= pd.to_datetime('2025-03-31')) & (df['Type'] == 'Cost')]\n\nresult = df_filtered['Amount'].sum()\n\nprint(f'The total cost between January 1st 2025 and March 31st 2025 is {result:,} UGX')"}`


The total cost between January 1st 2025 and March 31st 2025 is 15,332,800 UGX
The total cost between January 1st 2025 and March 31st 2025 is 15,332,800 UGX

> Finished chain.
The total cost between January 1st 2025 and March 31st 2025 is 15,332,800 UGX
